<a href="https://colab.research.google.com/github/sean-halpin/LSTM_text_generation/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


In [3]:
import requests
url = "http://www.gutenberg.org/cache/epub/11/pg11.txt"
r = requests.get(url, allow_redirects=True)

In [4]:
print(r.text.splitlines()[0])


﻿Project Gutenberg's Alice's Adventures in Wonderland, by Lewis Carroll


In [5]:
raw_text = r.text.lower()

In [6]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
print(char_to_int)

{'\n': 0, '\r': 1, ' ': 2, '!': 3, '"': 4, '#': 5, '$': 6, '%': 7, "'": 8, '(': 9, ')': 10, '*': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '?': 28, '@': 29, '[': 30, ']': 31, '_': 32, 'a': 33, 'b': 34, 'c': 35, 'd': 36, 'e': 37, 'f': 38, 'g': 39, 'h': 40, 'i': 41, 'j': 42, 'k': 43, 'l': 44, 'm': 45, 'n': 46, 'o': 47, 'p': 48, 'q': 49, 'r': 50, 's': 51, 't': 52, 'u': 53, 'v': 54, 'w': 55, 'x': 56, 'y': 57, 'z': 58, '\ufeff': 59}


In [8]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  167516
Total Vocab:  60


In [9]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  167416


In [10]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [11]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1308/1308 [==============================] - 53s 35ms/step - loss: 2.9848

Epoch 00001: loss improved from inf to 2.98478, saving model to weights-improvement-01-2.9848.hdf5
Epoch 2/20
1308/1308 [==============================] - 46s 35ms/step - loss: 2.7882

Epoch 00002: loss improved from 2.98478 to 2.78821, saving model to weights-improvement-02-2.7882.hdf5
Epoch 3/20
1308/1308 [==============================] - 46s 35ms/step - loss: 2.6880

Epoch 00003: loss improved from 2.78821 to 2.68797, saving model to weights-improvement-03-2.6880.hdf5
Epoch 4/20
1308/1308 [==============================] - 46s 35ms/step - loss: 2.6000

Epoch 00004: loss improved from 2.68797 to 2.59997, saving model to weights-improvement-04-2.6000.hdf5
Epoch 5/20
1308/1308 [==============================] - 46s 35ms/step - loss: 2.5508

Epoch 00005: loss improved from 2.59997 to 2.55080, saving model to weights-improvement-05-2.5508.hdf5
Epoch 6/20
1308/1308 [==============================] - 46s

In [22]:
import sys
# load the network weights
filename = "weights-improvement-11-2.2964.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" turtles, salmon, and so on;
then, when you've cleared all the jelly-fish out of the way--'

'that "
 you dan to tee to the tooee to the toier,' said the caterpillar.

'the mort toin i cen to toen ' said the caterpillar an an anl aoo tooe
that she wooee to the woeke and the tasten  the was so the tooee to the
coone th the tooee  the woote whit toe toete to the tooee to the tooee
ant the woite woat she woote whit the woite oo the tooee to the woiee
tht soee an the toote  the woote whit the tooee to the tooee to the houte
and the toree to the tooee to the tooee an the woike and the tasten 
and the tooee to the tooee to the tooee to the tooee  tha wasten an 
the woote  
Done.
